In [1]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = "maximal-park-424812-p9"

In [2]:
from google.cloud import bigquery
bq = bigquery.Client()

In [3]:
%load_ext google.cloud.bigquery

/home/macbook/p8_ajmehta_bkumar5/venv/lib/python3.12/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [4]:
%%bigquery geo_id_one  --project=maximal-park-424812-p9
SELECT geo_id 
FROM `bigquery-public-data.geo_us_boundaries.counties`
WHERE county_name = "Dane"

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
#q1
geo_id_one.iloc[0]['geo_id']

'55025'

In [6]:
%%bigquery counties_per_state --project=maximal-park-424812-p9
SELECT state_fips_code, COUNT(DISTINCT geo_id) AS num_counties
FROM `bigquery-public-data.geo_us_boundaries.counties`
GROUP BY state_fips_code
ORDER BY num_counties DESC
LIMIT 5



Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
#q2
# Convert np.int64 to Python int
counties_dict = {k: int(v) for k, v in zip(counties_per_state['state_fips_code'], counties_per_state['num_counties'])}
counties_dict


{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [8]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
q1 = bq.query("""
SELECT geo_id 
FROM `bigquery-public-data.geo_us_boundaries.counties`
WHERE county_name = "Dane"
""", job_config=no_cache)
df = q1.to_dataframe()
df

,geo_id
0,55025


In [9]:
q1_result_mb = q1.total_bytes_billed / 1024**2 # MB
q1_result = f"{int(q1_result_mb)} MB"

In [10]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
q2 = bq.query("""
SELECT state_fips_code, COUNT(DISTINCT geo_id) AS num_counties
FROM `bigquery-public-data.geo_us_boundaries.counties`
GROUP BY state_fips_code
ORDER BY num_counties DESC
LIMIT 5
""", job_config=no_cache)
df = q2.to_dataframe()
df

,state_fips_code,num_counties
0,48,254
1,13,159
2,51,133
3,21,120
4,29,115


In [11]:
q2_result_mb = q2.total_bytes_billed / 1024**2  # MB
q2_result = f"{int(q2_result_mb)} MB"  # Convert to integer and format

In [12]:
#q3
result = {
    "q1": q1_result,
    "q2": q2_result
}

result

{'q1': '10 MB', 'q2': '10 MB'}

In [13]:
project = "maximal-park-424812-p9" 

In [14]:
ds = bigquery.Dataset(f"{project}.p8")
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('maximal-park-424812-p9', 'p8'))

In [15]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
source = "https://storage.cloud.google.com/cs544_p8_ajmehta_bkumar5/hdma-wi-2021.parquet"
dataset = "p8"
job = bq.load_table_from_uri(source, f"{project}.{dataset}.hdma", job_config=config)
job.result()


LoadJob<project=maximal-park-424812-p9, location=US, id=f86af430-c2f9-497d-9934-1a0cd41675ff>

In [16]:
#q4
[ds.dataset_id for ds in bq.list_datasets(project)]

['p8']

In [17]:
%%bigquery loans_df
SELECT c.county_name, COUNT(h.action_taken) AS loan_count
FROM p8.hdma h
JOIN bigquery-public-data.geo_us_boundaries.counties c
ON h.county_code = c.county_fips_code
GROUP BY c.county_name
ORDER BY loan_count DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
#q5
dict_list = loans_df.to_dict(orient='records')

loans_dict = {}

for entry in dict_list:
    loans_dict[entry['county_name']] = entry['loan_count']

loans_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [19]:
url = "https://docs.google.com/spreadsheets/d/13e14LzDDm9U4y2KddlKFAy7exNdbo1OwJa-OTe4ywiw/edit?usp=sharing"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(f"{project}.{dataset}.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [20]:
%%bigquery income_df
SELECT count(*) AS my_income_count
FROM p8.applications
WHERE income = 250000

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
#q6
int(income_df['my_income_count'][0])

9

In [22]:
%%bigquery
SELECT * FROM `p8.applications`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,Timestamp,loan_amount,income,latitude,longitude
0,2024-11-26 20:31:58.477000+00:00,30000,100000,45.233277,-88.027144
1,2024-12-02 10:02:19.989000+00:00,1000000,250000,45.000000,45.000000
2,2024-12-02 10:02:44.188000+00:00,100000,9,47.323120,-123.120000
3,2024-12-02 10:03:17.986000+00:00,10000000,700000,-35.000000,100.000000
4,2024-12-02 10:03:22.677000+00:00,100000,40000,37.000000,42.000000


In [23]:
%%bigquery
CREATE OR REPLACE MODEL `maximal-park-424812-p9.p8.linear_reg_model`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['y']) AS
SELECT income AS x, loan_amount AS y
FROM `maximal-park-424812-p9.p8.applications`


Query is running:   0%|          |

""


In [24]:
%%bigquery r2_score_df --project=maximal-park-424812-p9
SELECT r2_score
FROM ML.EVALUATE(MODEL `maximal-park-424812-p9.p8.linear_reg_model`, (
    SELECT income AS x, loan_amount AS y
    FROM `maximal-park-424812-p9.p8.applications`
))


Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
#q7
r2_score = r2_score_df.iloc[0]['r2_score']  # Extract the R^2 score
r2_score_float = float(r2_score)  # Convert to a Python float
r2_score_float


0.02643986942442078

In [26]:
%%bigquery closest_distance_meters
SELECT
  MIN(ST_DISTANCE(ST_GEOGPOINT(longitude, latitude), ST_GEOGPOINT(-89.3842, 43.0747))) AS closest_distance_meters
FROM `maximal-park-424812-p9.p8.applications`



Query is running:   0%|          |

Downloading:   0%|          |

In [27]:
#q8
closest_distance_meters = closest_distance_meters.iloc[0]['closest_distance_meters']  
closest_distance_meters = float(closest_distance_meters)  # Convert to a Python float
closest_distance_meters

7.735784105240923

In [28]:
%%bigquery
SELECT county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'

Query is running:   0%|          |

Downloading:   0%|          |

,county_geom
0,"POLYGON((-90.925417 46.587301, -90.924712 46.5..."
1,"POLYGON((-87.432419 45.111772, -87.439231 45.0..."
2,"POLYGON((-91.166006 44.335102, -91.165998 44.3..."
3,"POLYGON((-90.668478 43.466273, -90.668478 43.4..."
4,"POLYGON((-92.760909 45.81664, -92.759938 45.81..."
...,...
67,"POLYGON((-89.012343 43.023489, -89.012346 43.0..."
68,"POLYGON((-90.316574 44.953278, -90.316576 44.9..."
69,"POLYGON((-90.042787 45.4676, -90.04279 45.4667..."
70,"POLYGON((-88.69812 42.842634, -88.698507 42.84..."


In [29]:
%%bigquery applications_df
SELECT c.county_name, COUNT(*) AS application_count
FROM p8.applications a
JOIN bigquery-public-data.geo_us_boundaries.counties c
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude))
WHERE c.state_fips_code = '55'
GROUP BY c.county_name

Query is running:   0%|          |

Downloading:   0%|          |

In [30]:
#q9
dict_list = applications_df.to_dict(orient='records')

applications_dict = {}

for entry in dict_list:
    applications_dict[entry['county_name']] = entry['application_count']

applications_dict

{'Dane': 34,
 'Marinette': 3,
 'Milwaukee': 4,
 'Brown': 1,
 'Price': 2,
 'Jefferson': 3,
 'Rock': 1,
 'Oneida': 2,
 'Door': 4,
 'Waushara': 1,
 'Juneau': 2,
 'Fond du Lac': 2,
 'Douglas': 1,
 'Green': 1,
 'Sawyer': 1,
 'Wood': 1,
 'Winnebago': 1,
 'Bayfield': 2,
 'Barron': 1,
 'Shawano': 1,
 'Taylor': 1}

In [31]:
%%bigquery bordering_counties_df
WITH dane_county AS (
  SELECT county_geom
  FROM bigquery-public-data.geo_us_boundaries.counties
  WHERE county_name = 'Dane' AND state_fips_code = '55'
)
SELECT c.county_name
FROM bigquery-public-data.geo_us_boundaries.counties c
JOIN dane_county d
ON ST_TOUCHES(c.county_geom, d.county_geom)
WHERE c.state_fips_code = '55' AND c.county_name != 'Dane'
ORDER BY c.county_name;




Query is running:   0%|          |

Downloading:   0%|          |

In [32]:
#q10
# Assuming the result of the query is stored in applications_df
bordering_counties = bordering_counties_df['county_name'].tolist()

# Sort the list alphabetically
bordering_counties.sort()

# Display the list of bordering counties
bordering_counties



['Columbia', 'Dodge', 'Green', 'Iowa', 'Jefferson', 'Rock', 'Sauk']